# **Tugas Group Project #2 (TGP #2): Klasifikasi**

## Impelementasi dan Eksperimen Prediksi Risiko Keuangan menggunakan Python

Kelompok 7 :


1.   Nicholas Evan S | 5026231146
2.   Kayla Putri M | 5-26231158
3. Tahiyyah Mufhimah | 5026231170
4. Baqhiz Faruq S | 5026231212



In [ ]:
import os
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import KFold, StratifiedKFold, RepeatedKFold, RepeatedStratifiedKFold, ShuffleSplit, StratifiedShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler

In [ ]:

# Load the CSV file
df_read = pd.read_csv("/content/data_home_credit.csv")
df = df_read.sample(n=2000, random_state=42)

df

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_QRT
38094,144113,0,0,157500.0,225000.0,8613.0,225000.0,0.022625,-12612.0,-2097.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
38479,144554,0,0,162000.0,835380.0,36796.5,675000.0,0.072508,-19376.0,-2731.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
53561,162045,0,2,225000.0,270000.0,13500.0,270000.0,0.018209,-12500.0,-2689.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
48055,155657,0,0,112500.0,640080.0,22990.5,450000.0,0.032561,-22750.0,-4240.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
79894,192613,0,0,225000.0,1096020.0,52857.0,900000.0,0.035792,-19572.0,365243.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53742,162256,0,0,135000.0,292500.0,11137.5,292500.0,0.020713,-17820.0,-1406.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
1815,102127,0,1,112500.0,454500.0,19255.5,454500.0,0.028663,-21244.0,365243.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
55637,164466,0,0,315000.0,135000.0,6750.0,135000.0,0.010006,-18012.0,-1468.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3799,104440,0,2,135000.0,835380.0,40320.0,675000.0,0.025164,-12200.0,-3499.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


#1 - Pemilahan Data

### Tujuan
Membagi data yang sudah disiapkan dalam TGP #1 menjadi data pelatihan dan data tes.

### Langkah-Langkah
1. **Pembagian Data**
   - Bagi keseluruhan data menjadi data pelatihan sebanyak 75% dan data tes sebanyak 25%.
   - Baik untuk data pelatihan maupun data tes harus dibagi secara proporsional untuk masing-masing kelas label menggunakan *stratified sampling*.
   - Tunjukkan statistik dari hasil pembagian data tersebut.

2. **10-fold cross validation**
   - Untuk memperoleh model klasifikasi terbaik dari beberapa metode klasifikasi yang diimplementasikan, gunakan keseluruhan data pelatihan dengan menggunakan *10-fold cross validation*.
   - Gunakan metode *stratified sampling* dalam menentukan pembagian data dalam *10-fold cross validation*.

3. **Pengujian Kinerja Model**
   - Gunakan data tes hanya untuk menguji kinerja model klasifikasi terbaik berdasarkan parameter terbaik yang diperoleh dari *10-fold cross validation*.

4. **Transformasi Nilai Atribut**
   - Lakukan transformasi nilai domain dari setiap atribut (jika diperlukan) untuk menyesuaikan dengan metode prediksi yang digunakan dalam eksperimen.


###1.1 Pembagian Data

In [ ]:
# Pilih fitur dan target
X = df[['AMT_INCOME_TOTAL', 'AMT_CREDIT', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']]
y = df['TARGET'].astype(str)  # Ubah ke string jika perlu untuk proses balancing

print(f"Jumlah fitur: {X.shape[1]}")
print(f"Jumlah target: {y.shape[0]}")

Jumlah fitur: 7
Jumlah target: 2000


In [ ]:
# Tangani nilai NaN (wajib sebelum modeling)
X = X.fillna(X.mean())

# Balancing class label (karena TARGET = 1 jauh lebih sedikit)
undersampler = RandomUnderSampler(random_state=42)
X_bal, y_bal = undersampler.fit_resample(X, y)

# Split data secara stratified
X_train, X_test, y_train, y_test = train_test_split(
    X_bal, y_bal, test_size=0.25, random_state=42, stratify=y_bal
)

# Tampilkan ukuran data training dan testing
print(f"\nUkuran Data Training: {X_train.shape}")
print(f"Ukuran Data Testing: {X_test.shape}")

# Distribusi label
print("\nDistribusi Label pada Training Set:")
print(y_train.value_counts(normalize=True))

print("\nDistribusi Label pada Test Set:")
print(y_test.value_counts(normalize=True))


Ukuran Data Training: (228, 7)
Ukuran Data Testing: (76, 7)

Distribusi Label pada Training Set:
TARGET
1    0.5
0    0.5
Name: proportion, dtype: float64

Distribusi Label pada Test Set:
TARGET
0    0.5
1    0.5
Name: proportion, dtype: float64


###1.2 10-fold cross validation

In [ ]:
# Inisialisasi StratifiedKFold dengan 10 fold
strat_kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=130)

# Variabel untuk menyimpan indeks data per fold
train_indices = []
test_indices = []

# Loop untuk membagi data dan menyimpan indeks
for fold, (train_index, test_index) in enumerate(strat_kfold.split(X, y)):
    print(f"Fold {fold + 1}")
    print(f"  Train: {len(train_index)} rows")
    print(f"  Test: {len(test_index)} rows")

    # Simpan indeks dari setiap fold
    train_indices.append(train_index)
    test_indices.append(test_index)

Fold 1
  Train: 1800 rows
  Test: 200 rows
Fold 2
  Train: 1800 rows
  Test: 200 rows
Fold 3
  Train: 1800 rows
  Test: 200 rows
Fold 4
  Train: 1800 rows
  Test: 200 rows
Fold 5
  Train: 1800 rows
  Test: 200 rows
Fold 6
  Train: 1800 rows
  Test: 200 rows
Fold 7
  Train: 1800 rows
  Test: 200 rows
Fold 8
  Train: 1800 rows
  Test: 200 rows
Fold 9
  Train: 1800 rows
  Test: 200 rows
Fold 10
  Train: 1800 rows
  Test: 200 rows


In [ ]:
X.iloc[train_indices[0]].describe()

,AMT_INCOME_TOTAL,AMT_CREDIT,DAYS_BIRTH,DAYS_EMPLOYED,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3
count,1.800000e+03,1.800000e+03,1800.000000,1800.000000,1800.000000,1800.000000,1800.000000
mean,2.334723e+05,5.946848e+05,-16117.408889,67411.495556,0.505550,0.517946,0.515978
std,2.755666e+06,3.994175e+05,4423.714071,144302.690071,0.130809,0.192238,0.176755
min,2.857500e+04,4.500000e+04,-25097.000000,-16121.000000,0.041376,0.000086,0.000527
25%,1.125000e+05,2.700000e+05,-19846.000000,-2732.250000,0.505998,0.403467,0.406617
50%,1.480500e+05,5.120640e+05,-15940.000000,-1288.000000,0.505998,0.570372,0.535276
75%,2.025000e+05,8.086500e+05,-12418.000000,-262.500000,0.505998,0.667640,0.643026
max,1.170000e+08,2.700000e+06,-7696.000000,365243.000000,0.951624,0.817368,0.863363


In [ ]:
X_bal.describe()

,AMT_INCOME_TOTAL,AMT_CREDIT,DAYS_BIRTH,DAYS_EMPLOYED,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3
count,3.040000e+02,3.040000e+02,304.000000,304.000000,304.000000,304.000000,304.000000
mean,5.489923e+05,5.921097e+05,-15775.190789,56930.921053,0.484291,0.457718,0.486399
std,6.701702e+06,3.968511e+05,4496.696920,135389.908214,0.147889,0.209077,0.189328
min,4.500000e+04,6.750000e+04,-24720.000000,-13905.000000,0.041376,0.000086,0.000527
25%,1.096875e+05,2.700000e+05,-19855.500000,-2625.750000,0.505998,0.290558,0.348646
50%,1.350000e+05,5.179815e+05,-15484.500000,-1276.500000,0.505998,0.510469,0.535276
75%,1.856250e+05,8.100000e+05,-12106.750000,-378.000000,0.505998,0.623872,0.613559
max,1.170000e+08,2.695500e+06,-7760.000000,365243.000000,0.906866,0.805339,0.896010


In [ ]:
scaler = StandardScaler()

# Scale AMT_INCOME_TOTAL
X_train['AMT_INCOME_TOTAL'] = scaler.fit_transform(X_train[['AMT_INCOME_TOTAL']])
X_test['AMT_INCOME_TOTAL'] = scaler.transform(X_test[['AMT_INCOME_TOTAL']])

# Scale AMT_CREDIT
X_train['AMT_CREDIT'] = scaler.fit_transform(X_train[['AMT_CREDIT']])
X_test['AMT_CREDIT'] = scaler.transform(X_test[['AMT_CREDIT']])

# Scale DAYS_BIRTH
X_train['DAYS_BIRTH'] = scaler.fit_transform(X_train[['DAYS_BIRTH']])
X_test['DAYS_BIRTH'] = scaler.transform(X_test[['DAYS_BIRTH']])

# Scale DAYS_EMPLOYED
X_train['DAYS_EMPLOYED'] = scaler.fit_transform(X_train[['DAYS_EMPLOYED']])
X_test['DAYS_EMPLOYED'] = scaler.transform(X_test[['DAYS_EMPLOYED']])

# Scale EXT_SOURCE_1
X_train['EXT_SOURCE_1'] = scaler.fit_transform(X_train[['EXT_SOURCE_1']])
X_test['EXT_SOURCE_1'] = scaler.transform(X_test[['EXT_SOURCE_1']])

# Scale EXT_SOURCE_2
X_train['EXT_SOURCE_2'] = scaler.fit_transform(X_train[['EXT_SOURCE_2']])
X_test['EXT_SOURCE_2'] = scaler.transform(X_test[['EXT_SOURCE_2']])

# Scale EXT_SOURCE_3
X_train['EXT_SOURCE_3'] = scaler.fit_transform(X_train[['EXT_SOURCE_3']])
X_test['EXT_SOURCE_3'] = scaler.transform(X_test[['EXT_SOURCE_3']])

# Lihat ringkasan statistik setelah scaling
print(X_train.describe())

       AMT_INCOME_TOTAL    AMT_CREDIT    DAYS_BIRTH  DAYS_EMPLOYED  \
count      2.280000e+02  2.280000e+02  2.280000e+02   2.280000e+02   
mean       3.895519e-18  5.453727e-17 -1.752984e-16   1.947760e-17   
std        1.002200e+00  1.002200e+00  1.002200e+00   1.002200e+00   
min       -8.109240e-02 -1.285290e+00 -1.949362e+00  -5.073097e-01   
25%       -7.409888e-02 -7.915874e-01 -9.155477e-01  -4.201464e-01   
50%       -7.001933e-02 -1.966819e-01  8.398827e-02  -4.106016e-01   
75%       -6.419140e-02  5.257674e-01  8.160642e-01  -4.040907e-01   
max        1.506511e+01  5.136432e+00  1.818306e+00   2.388381e+00   

       EXT_SOURCE_1  EXT_SOURCE_2  EXT_SOURCE_3  
count  2.280000e+02  2.280000e+02  2.280000e+02  
mean  -5.453727e-16 -8.570143e-17 -3.272236e-16  
std    1.002200e+00  1.002200e+00  1.002200e+00  
min   -3.097791e+00 -2.235920e+00 -2.552019e+00  
25%    1.841669e-01 -7.725157e-01 -7.306806e-01  
50%    1.841669e-01  2.402447e-01  2.196204e-01  
75%    1.841669e-01

#2 - Eksperimen dengan Model Klasifikasi k-NN

### Tujuan
Mengimplementasikan metode klasifikasi k-NN dan melakukan analisis dari hasil eksperimen dengan k-NN.

### Langkah-Langkah
1. **Eksperimen Variasi Nilai k**
   - Lakukan eksperimen untuk memperoleh model klasifikasi k-NN terbaik menggunakan variasi nilai k sebanyak 10 kali.
   - Anda bebas menentukan variasi nilai k, tetapi pastikan nilainya tidak terlalu kecil atau terlalu besar.

2. **Validasi dengan 10-Fold Cross Validation**
   - Untuk setiap nilai k yang berbeda, jalankan model k-NN menggunakan *10-fold cross validation*.
   - Setiap nilai k akan menghasilkan lima hasil validasi dari model.

3. **Pengukuran Kinerja Model**
   - Hitung akurasi, presisi, *recall*, F1-score, dan AUC ROC dari setiap data validasi pada langkah ke-2.

4. **Analisis Kinerja Model**
   - Lakukan analisis dari kelima hasil kinerja model k-NN untuk nilai k yang sedang diuji berdasarkan hasil pada langkah ke-3.

10. **Penentuan Model Terbaik**
   - Berdasarkan analisis pada langkah ke-4, tentukan nilai k yang menghasilkan model terbaik.
   - Jalankan model k-NN pada keseluruhan data pelatihan menggunakan nilai k tersebut dan hitung kinerja model (akurasi, presisi, *recall*, F1-score, dan AUC ROC).

6. **Ulangi untuk Nilai k Berikutnya**
   - Ulangi langkah ke-2 hingga ke-10 untuk nilai k berikutnya.

7. **Analisis Hasil Kinerja Model**
   - Lakukan analisis dari hasil kinerja model k-NN dari ke sepuluh variasi nilai k yang diperoleh pada langkah ke-6.

8. **Prediksi dengan Model Terbaik**
   - Gunakan model k-NN terbaik untuk memprediksi semua record dari data tes.
   - Hitung akurasi, presisi, *recall*, F1-score, dan AUC ROC dari model terbaik pada data tes (data yang tidak digunakan dalam membangun model terbaik).


### 2.1 Eksperimen Variasi Nilai k

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
# Tentukan variasi nilai k untuk eksperimen
k_values = [3, 5, 7, 9, 11, 13, 15, 17, 19, 21]

# Variabel untuk menyimpan model hasil pelatihan
knn_trained_models = {}  # Akan menyimpan model untuk setiap nilai k

# Loop untuk setiap nilai k
for k in k_values:
    print(f"\nEvaluating k = {k}")

    # Variabel untuk menyimpan model per fold untuk k saat ini
    models_per_k = []

    # Loop untuk melatih model dengan 10-Fold Cross Validation
    for fold, (train_index, test_index) in enumerate(zip(train_indices, test_indices), start=1):
        # Membagi data menjadi data pelatihan berdasarkan fold saat ini
        X_train_fold, y_train_fold = X.iloc[train_index], y.iloc[train_index]

        # Inisialisasi model k-NN dengan nilai k tertentu
        model_knn = KNeighborsClassifier(n_neighbors=k)

        # Melatih model
        model_knn.fit(X_train_fold, y_train_fold)

        # Simpan model yang sudah dilatih
        models_per_k.append(model_knn)

        # Menampilkan status pelatihan model
        print(f"Fold {fold} - Training completed")

    # Simpan model untuk setiap nilai k
    knn_trained_models[k] = models_per_k


Evaluating k = 3
Fold 1 - Training completed
Fold 2 - Training completed
Fold 3 - Training completed
Fold 4 - Training completed
Fold 5 - Training completed
Fold 6 - Training completed
Fold 7 - Training completed
Fold 8 - Training completed
Fold 9 - Training completed
Fold 10 - Training completed

Evaluating k = 5
Fold 1 - Training completed
Fold 2 - Training completed
Fold 3 - Training completed
Fold 4 - Training completed
Fold 5 - Training completed
Fold 6 - Training completed
Fold 7 - Training completed
Fold 8 - Training completed
Fold 9 - Training completed
Fold 10 - Training completed

Evaluating k = 7
Fold 1 - Training completed
Fold 2 - Training completed
Fold 3 - Training completed
Fold 4 - Training completed
Fold 5 - Training completed
Fold 6 - Training completed
Fold 7 - Training completed
Fold 8 - Training completed
Fold 9 - Training completed
Fold 10 - Training completed

Evaluating k = 9
Fold 1 - Training completed
Fold 2 - Training completed
Fold 3 - Training completed
F

### 2.2 Validasi dengan 10-Fold Cross Validation

In [ ]:
# Variabel untuk menyimpan prediksi dan probabilitas
y_preds = {}  # Menyimpan prediksi untuk setiap nilai k dan fold
y_probs = {}  # Menyimpan probabilitas untuk setiap nilai k dan fold

# Loop untuk setiap nilai k (melanjutkan dari model yang sudah dilatih)
for k, models_per_k in knn_trained_models.items():
    print(f"\nGenerating predictions for k = {k}")

    # Variabel untuk menyimpan prediksi dan probabilitas per fold
    k_y_preds = []
    k_y_probs = []
    # Loop untuk setiap fold dan model yang sudah dilatih
    for fold, (train_index, test_index) in enumerate(zip(train_indices, test_indices), start=1):
        # Ambil model k-NN yang sudah dilatih untuk fold ini
        model_knn = models_per_k[fold - 1]

        # Ambil data testing untuk fold saat ini
        X_test_fold = X.iloc[test_index]
        y_test_fold = y.iloc[test_index]

        # Melakukan prediksi pada data pengujian
        y_pred = model_knn.predict(X_test_fold)
        k_y_preds.append(y_pred)

        # Ambil probabilitas prediksi
        y_prob = model_knn.predict_proba(X_test_fold)
        k_y_probs.append(y_prob)

        # Tampilkan status prediksi untuk fold ini
        print(f"Fold {fold} - Predictions completed")

    # Simpan prediksi dan probabilitas untuk nilai k saat ini
    y_preds[k] = k_y_preds
    y_probs[k] = k_y_probs



Generating predictions for k = 3
Fold 1 - Predictions completed
Fold 2 - Predictions completed
Fold 3 - Predictions completed
Fold 4 - Predictions completed
Fold 5 - Predictions completed
Fold 6 - Predictions completed
Fold 7 - Predictions completed
Fold 8 - Predictions completed
Fold 9 - Predictions completed
Fold 10 - Predictions completed

Generating predictions for k = 5
Fold 1 - Predictions completed
Fold 2 - Predictions completed
Fold 3 - Predictions completed
Fold 4 - Predictions completed
Fold 5 - Predictions completed
Fold 6 - Predictions completed
Fold 7 - Predictions completed
Fold 8 - Predictions completed
Fold 9 - Predictions completed
Fold 10 - Predictions completed

Generating predictions for k = 7
Fold 1 - Predictions completed
Fold 2 - Predictions completed
Fold 3 - Predictions completed
Fold 4 - Predictions completed
Fold 5 - Predictions completed
Fold 6 - Predictions completed
Fold 7 - Predictions completed
Fold 8 - Predictions completed
Fold 9 - Predictions complet

### 2.3 Pengukuran Kinerja Model

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.preprocessing import LabelBinarizer
import pandas as pd

# Variabel untuk menyimpan hasil kinerja
results = {
    'k': [], 'fold': [], 'accuracy': [], 'precision': [],
    'recall': [], 'f1_score': [], 'roc_auc': []
}

# Loop untuk setiap nilai k
for k, folds_preds in y_preds.items():
    print(f"\nEvaluating performance for k = {k}")

    # Loop untuk setiap fold
    for fold, (train_index, test_index) in enumerate(zip(train_indices, test_indices), start=1):
        y_test_fold = y.iloc[test_index]
        y_pred = folds_preds[fold - 1]
        y_prob = y_probs[k][fold - 1]

        # Hitung metrik kinerja
        accuracy = accuracy_score(y_test_fold, y_pred)
        precision = precision_score(y_test_fold, y_pred, average='macro', zero_division=0)
        recall = recall_score(y_test_fold, y_pred, average='macro', zero_division=0)
        f1 = f1_score(y_test_fold, y_pred, average='macro', zero_division=0)

        # Hitung ROC AUC untuk binary atau multi-class
        roc_auc = None
        try:
            lb = LabelBinarizer()
            y_test_bin = lb.fit_transform(y_test_fold)

            if y_test_bin.shape[1] == 1:  # Binary classification
                roc_auc = roc_auc_score(y_test_bin, y_prob[:, 1])
            else:  # Multi-class case
                roc_auc = roc_auc_score(
                    y_test_bin, y_prob, multi_class='ovr', average='weighted'
                )
        except Exception as e:
            print(f"Warning: Could not calculate ROC AUC for fold {fold}: {str(e)}")

        # Simpan hasil kinerja
        results['k'].append(k)
        results['fold'].append(fold)
        results['accuracy'].append(accuracy)
        results['precision'].append(precision)
        results['recall'].append(recall)
        results['f1_score'].append(f1)
        results['roc_auc'].append(roc_auc)

        # Tampilkan hasil per fold
        roc_auc_display = f"{roc_auc:.4f}" if roc_auc is not None else "Not Applicable"
        print(f"Fold {fold} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, "
              f"Recall: {recall:.4f}, F1 Score: {f1:.4f}, ROC AUC: {roc_auc_display}")

# Konversi hasil ke DataFrame untuk analisis lebih lanjut
results_df = pd.DataFrame(results)


Evaluating performance for k = 3
Fold 1 - Accuracy: 0.9200, Precision: 0.4623, Recall: 0.4973, F1 Score: 0.4792, ROC AUC: 0.4744
Fold 2 - Accuracy: 0.8950, Precision: 0.4613, Recall: 0.4838, F1 Score: 0.4723, ROC AUC: 0.5479
Fold 3 - Accuracy: 0.9100, Precision: 0.4619, Recall: 0.4919, F1 Score: 0.4764, ROC AUC: 0.4957
Fold 4 - Accuracy: 0.9150, Precision: 0.5893, Recall: 0.5252, F1 Score: 0.5303, ROC AUC: 0.5757
Fold 5 - Accuracy: 0.9100, Precision: 0.4619, Recall: 0.4919, F1 Score: 0.4764, ROC AUC: 0.5474
Fold 6 - Accuracy: 0.9150, Precision: 0.4621, Recall: 0.4946, F1 Score: 0.4778, ROC AUC: 0.4519
Fold 7 - Accuracy: 0.9000, Precision: 0.4615, Recall: 0.4865, F1 Score: 0.4737, ROC AUC: 0.5324
Fold 8 - Accuracy: 0.9150, Precision: 0.4621, Recall: 0.4946, F1 Score: 0.4778, ROC AUC: 0.4941
Fold 9 - Accuracy: 0.8850, Precision: 0.4585, Recall: 0.4810, F1 Score: 0.4695, ROC AUC: 0.5088
Fold 10 - Accuracy: 0.9250, Precision: 0.9623, Recall: 0.5312, F1 Score: 0.5392, ROC AUC: 0.6270

Eval

### 2.4 Training pada model k terbaik

In [ ]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

# 1. Temukan kombinasi terbaik berdasarkan rata-rata accuracy
mean_accuracy = results_df.groupby('k')['accuracy'].mean().reset_index()
best_combination = mean_accuracy.loc[mean_accuracy['accuracy'].idxmax()]
# 2. Filter data berdasarkan k terbaik
best_k = int(best_combination['k'])
# Ambil semua hasil fold untuk k terbaik
best_model_folds = results_df[results_df['k'] == best_k]
# 3. Temukan fold terbaik berdasarkan accuracy tertinggi
best_fold = best_model_folds.loc[best_model_folds['accuracy'].idxmax()]
best_fold_number = best_fold['fold']  # Dapatkan nomor fold terbaik

# 4. Tampilkan hasil analisis
print("Kombinasi Terbaik Berdasarkan Accuracy:")
print(f"k: {best_k}, Mean Accuracy: {best_combination['accuracy']:.4f}")

print(f"\nFold Terbaik dari Kombinasi Terbaik:")
print(f"Fold: {best_fold['fold']}, Accuracy: {best_fold['accuracy']:.4f}, "
      f"Precision: {best_fold['precision']:.4f}, Recall: {best_fold['recall']:.4f}, "
      f"F1 Score: {best_fold['f1_score']:.4f}, ROC AUC: {best_fold['roc_auc']:.4f}")

# 5. Ambil model terbaik dari fold terbaik dan kombinasi terbaik
print(f"\nFold terbaik berada pada Fold {best_fold_number} dengan k = {best_k}.")

# 6. Standarisasi ulang data untuk training model terbaik pada seluruh data train
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 7. Inisialisasi model k-NN dengan k terbaik
final_model_knn = KNeighborsClassifier(n_neighbors=best_k)

# 8. Latih model terbaik menggunakan seluruh data train
final_model_knn.fit(X_scaled, y)

Kombinasi Terbaik Berdasarkan Accuracy:
k: 7, Mean Accuracy: 0.9240

Fold Terbaik dari Kombinasi Terbaik:
Fold: 1.0, Accuracy: 0.9250, Precision: 0.4625, Recall: 0.5000, F1 Score: 0.4805, ROC AUC: 0.4369

Fold terbaik berada pada Fold 1.0 dengan k = 7.


KNeighborsClassifier(n_neighbors=7)

### 2.5 Pengukuran Kinerja Model k Terbaik

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np

# 1. Prediksi label untuk seluruh data menggunakan model terbaik
print("\nMelakukan prediksi dengan model k-NN terbaik...")
y_pred_new = final_model_knn.predict(X_scaled)
y_probs_new = final_model_knn.predict_proba(X_scaled)

# 2. Evaluasi kinerja pada seluruh data
accuracy_new = accuracy_score(y, y_pred_new)
precision_new = precision_score(y, y_pred_new, average='macro', zero_division=0)
recall_new = recall_score(y, y_pred_new, average='macro', zero_division=0)
f1_new = f1_score(y, y_pred_new, average='macro', zero_division=0)

# 3. Hitung ROC AUC
try:
    if len(np.unique(y)) > 2:  # Multiclass
        roc_auc_new = roc_auc_score(y, y_probs_new, multi_class='ovr', average='macro')
    else:  # Binary classification
        roc_auc_new = roc_auc_score(y, y_probs_new[:, 1])
except ValueError as e:
    roc_auc_new = None
    print(f"Error calculating ROC AUC on new data: {e}")

# 4. Cetak hasil evaluasi
print("\nEvaluasi Kinerja Model Terbaik pada Data:")
print(f"Accuracy: {accuracy_new:.4f}")
print(f"Precision: {precision_new:.4f}")
print(f"Recall: {recall_new:.4f}")
print(f"F1 Score: {f1_new:.4f}")

if roc_auc_new is not None:
    print(f"ROC AUC: {roc_auc_new:.4f}")
else:
    print("ROC AUC: Not Applicable")


Melakukan prediksi dengan model k-NN terbaik...

Evaluasi Kinerja Model Terbaik pada Data:
Accuracy: 0.9265
Precision: 0.7744
Recall: 0.5406
F1 Score: 0.5559
ROC AUC: 0.9010


#3 - Eksperimen dengan Model Naïve Bayes

### Tujuan
Mengimplementasikan metode klasifikasi berbasis Naïve Bayes dan melakukan analisis dari hasil eksperimen.

### Langkah-Langkah
1. **Eksperimen dengan 10-Fold Cross Validation**
   - Lakukan eksperimen untuk memperoleh model klasifikasi Naïve Bayes terbaik menggunakan lima kombinasi data pelatihan dan data validasi dari *10-fold cross validation*.

2. **Pengukuran Kinerja Model**
   - Hitung akurasi, presisi, *recall*, F1-score, dan AUC ROC dari setiap data validasi pada langkah ke-1.

3. **Analisis Kinerja Model**
   - Lakukan analisis dari ke sepuluh hasil kinerja model Naïve Bayes yang diperoleh dari eksperimen pada langkah ke-2.

4. **Kesimpulan dari Analisis Kinerja**
   - Lakukan analisis lebih lanjut dari hasil kinerja ke sepuluh model Naïve Bayes yang diuji pada langkah ke-3.

5. **Prediksi dengan Model Terbaik**
   - Gunakan model Naïve Bayes terbaik untuk memprediksi semua record dari data tes.
   - Hitung akurasi, presisi, *recall*, F1-score, dan AUC ROC dari model terbaik pada data tes (data yang sama sekali tidak digunakan dalam membangun model terbaik).


### 3.1 Eksperimen dengan 10-Fold Cross Validation

In [ ]:
# Variabel untuk menyimpan model hasil pelatihan
nb_trained_models = []

# Loop untuk melatih model
for fold, (train_index, test_index) in enumerate(zip(train_indices, test_indices)):
    # Membagi data menjadi data pelatihan dan data pengujian
    X_train_fold, y_train_fold = X.iloc[train_index], y.iloc[train_index]

    # Inisialisasi model Naïve Bayes
    model_nb = GaussianNB()

    # Melatih model
    model_nb.fit(X_train_fold, y_train_fold)

    # Simpan model yang sudah dilatih
    nb_trained_models.append(model_nb)

    # Menampilkan status pelatihan model
    print(f"Fold {fold + 1} - Model training complete")

Fold 1 - Model training complete
Fold 2 - Model training complete
Fold 3 - Model training complete
Fold 4 - Model training complete
Fold 5 - Model training complete
Fold 6 - Model training complete
Fold 7 - Model training complete
Fold 8 - Model training complete
Fold 9 - Model training complete
Fold 10 - Model training complete


###3.2 Validasi Model

In [ ]:
# Variabel untuk menyimpan prediksi dan probabilitas
y_preds = []
y_probs = []

# Loop untuk melakukan prediksi
for fold, (train_index, test_index) in enumerate(zip(train_indices, test_indices)):
    # Mengambil data pengujian
    X_test_fold = X.iloc[test_index]
    y_test_fold = y.iloc[test_index]

    # Mengambil model yang sudah dilatih di fold ini
    model_nb = nb_trained_models[fold]

    # Melakukan prediksi pada data pengujian
    y_pred = model_nb.predict(X_test_fold)
    y_preds.append(y_pred)

    # Mengambil probabilitas prediksi untuk semua kelas
    y_prob = model_nb.predict_proba(X_test_fold)

    # Menyimpan probabilitas
    y_probs.append(y_prob)

    print(f"Fold {fold + 1} - Predictions complete")

Fold 1 - Predictions complete
Fold 2 - Predictions complete
Fold 3 - Predictions complete
Fold 4 - Predictions complete
Fold 5 - Predictions complete
Fold 6 - Predictions complete
Fold 7 - Predictions complete
Fold 8 - Predictions complete
Fold 9 - Predictions complete
Fold 10 - Predictions complete


### 3.3 Analisis Kinerja Model

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Variabel untuk menyimpan hasil kinerja
results = {
    'fold': [],
    'accuracy': [],
    'precision': [],
    'recall': [],
    'f1_score': [],
    'roc_auc': []
}

# Loop untuk menghitung kinerja model
for fold, (train_index, test_index) in enumerate(zip(train_indices, test_indices)):
    # Mengambil data pengujian yang sesuai
    y_test_fold = y.iloc[test_index]
    y_pred = y_preds[fold]
    y_prob = y_probs[fold]  # Ambil probabilitas untuk fold ini

    # Menghitung metrik kinerja
    accuracy = accuracy_score(y_test_fold, y_pred)
    precision = precision_score(y_test_fold, y_pred, average='macro', zero_division=0)
    recall = recall_score(y_test_fold, y_pred, average='macro', zero_division=0)
    f1 = f1_score(y_test_fold, y_pred, average='macro', zero_division=0)

    # Menampilkan ukuran untuk debug
    print(f"Ukuran y_test: {y_test_fold.shape}, Ukuran y_probs: {y_prob.shape}")

    # Menghitung AUC ROC
    roc_auc = None
    num_classes = len(set(y_test_fold))  # Menghitung jumlah kelas

    if num_classes > 2:
        # Untuk lebih dari dua kelas, gunakan rata-rata 'macro' untuk ROC AUC
        try:
            roc_auc = roc_auc_score(y_test_fold, y_prob, multi_class='ovr')
        except ValueError as e:
            print(f"Error calculating ROC AUC for fold {fold + 1}: {e}")
    elif num_classes == 2:
        # Untuk dua kelas, gunakan ROC AUC biasa
        try:
            roc_auc = roc_auc_score(y_test_fold, y_prob[:, 1])  # Ambil probabilitas untuk kelas positif
        except ValueError as e:
            print(f"Error calculating ROC AUC for fold {fold + 1}: {e}")

    # Menyimpan hasil
    results['fold'].append(fold + 1)
    results['accuracy'].append(accuracy)
    results['precision'].append(precision)
    results['recall'].append(recall)
    results['f1_score'].append(f1)
    results['roc_auc'].append(roc_auc)

    # Mencetak hasil dengan penanganan None
    if roc_auc is not None:
        print(f"Fold {fold + 1} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}, ROC AUC: {roc_auc:.4f}, Number of Classes: {num_classes}")
    else:
        print(f"Fold {fold + 1} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}, ROC AUC: Not Applicable, Number of Classes: {num_classes}")


Ukuran y_test: (200,), Ukuran y_probs: (200, 2)
Fold 1 - Accuracy: 0.9200, Precision: 0.4623, Recall: 0.4973, F1 Score: 0.4792, ROC AUC: 0.4977, Number of Classes: 2
Ukuran y_test: (200,), Ukuran y_probs: (200, 2)
Fold 2 - Accuracy: 0.9200, Precision: 0.4623, Recall: 0.4973, F1 Score: 0.4792, ROC AUC: 0.5838, Number of Classes: 2
Ukuran y_test: (200,), Ukuran y_probs: (200, 2)
Fold 3 - Accuracy: 0.9250, Precision: 0.4625, Recall: 0.5000, F1 Score: 0.4805, ROC AUC: 0.4915, Number of Classes: 2
Ukuran y_test: (200,), Ukuran y_probs: (200, 2)
Fold 4 - Accuracy: 0.9200, Precision: 0.4623, Recall: 0.4973, F1 Score: 0.4792, ROC AUC: 0.6007, Number of Classes: 2
Ukuran y_test: (200,), Ukuran y_probs: (200, 2)
Fold 5 - Accuracy: 0.9250, Precision: 0.4625, Recall: 0.5000, F1 Score: 0.4805, ROC AUC: 0.5510, Number of Classes: 2
Ukuran y_test: (200,), Ukuran y_probs: (200, 2)
Fold 6 - Accuracy: 0.9250, Precision: 0.4625, Recall: 0.5000, F1 Score: 0.4805, ROC AUC: 0.6076, Number of Classes: 2
Ukur

### 3.4 Training dan Validasi Model Terbaik

In [ ]:
# Setelah semua fold dihitung, kita mencari fold terbaik berdasarkan F1 Score
best_fold_index = np.argmax(results['f1_score'])
print(f"Fold terbaik berdasarkan F1 Score adalah Fold {results['fold'][best_fold_index]} dengan nilai F1 Score {results['f1_score'][best_fold_index]:.4f}")

# Inisialisasi model
best_model = nb_trained_models[best_fold_index]

# Latih model menggunakan keseluruhan data
best_model = best_model.fit(X, y)

Fold terbaik berdasarkan F1 Score adalah Fold 3 dengan nilai F1 Score 0.4805


### 3.5 Pengukuran Model Terbaik

In [ ]:
# Prediksi label untuk data baru
y_pred_new = best_model.predict(X)
y_probs_new = best_model.predict_proba(X)

# Evaluasi kinerja pada data baru
accuracy_new = accuracy_score(y, y_pred_new)
precision_new = precision_score(y, y_pred_new, average='macro', zero_division=0)
recall_new = recall_score(y, y_pred_new, average='macro', zero_division=0)
f1_new = f1_score(y, y_pred_new, average='macro', zero_division=0)

try:
    roc_auc_new = roc_auc_score(y, y_probs_new, multi_class='ovr')
except ValueError as e:
    roc_auc_new = None
    print(f"Error calculating ROC AUC on new data: {e}")

# Cetak hasil evaluasi
print(f"Evaluasi pada data baru:")
print(f"Accuracy: {accuracy_new:.4f}")
print(f"Precision: {precision_new:.4f}")
print(f"Recall: {recall_new:.4f}")
print(f"F1 Score: {f1_new:.4f}")
if roc_auc_new is not None:
    print(f"ROC AUC: {roc_auc_new:.4f}")
else:
    print("ROC AUC: Not Applicable")

Error calculating ROC AUC on new data: y should be a 1d array, got an array of shape (2000, 2) instead.
Evaluasi pada data baru:
Accuracy: 0.9185
Precision: 0.5289
Recall: 0.5031
F1 Score: 0.4907
ROC AUC: Not Applicable


# 4 - Eksperimen dengan Model Logistic Regression

### Tujuan
Mengimplementasikan metode klasifikasi berbasis Logistic Regression dan melakukan analisis dari hasil eksperimen.

### Langkah-Langkah
1. **Eksperimen dengan 10-Fold Cross Validation**
   - Lakukan eksperimen untuk memperoleh model klasifikasi berbasis Logistic Regression terbaik menggunakan lima kombinasi data pelatihan dan data validasi dari *10-fold cross validation*.

2. **Pengukuran Kinerja Model**
   - Hitung akurasi, presisi, *recall*, F1-score, dan AUC ROC dari setiap data validasi pada langkah ke-1.

3. **Analisis Kinerja Model**
   - Lakukan analisis dari sepuluh hasil kinerja model Logistic Regression yang diperoleh dari eksperimen pada langkah ke-2.

4. **Kesimpulan dari Analisis Kinerja**
   - Lakukan analisis lebih lanjut dari hasil kinerja sepuluh model Logistic Regression yang diuji pada langkah ke-3.

5. **Prediksi dengan Model Terbaik**
   - Gunakan model Logistic Regression terbaik untuk memprediksi semua record dari data tes.
   - Hitung akurasi, presisi, *recall*, F1-score, dan AUC ROC dari model terbaik pada data tes (data yang sama sekali tidak digunakan dalam membangun model terbaik).


### 4.1 Eksperimen dengan 10-Fold Cross Validation

In [ ]:
from sklearn.linear_model import LogisticRegression

# Variabel untuk menyimpan model hasil pelatihan
logreg_trained_models = []  # Menyimpan model untuk setiap fold

# Loop untuk melatih model menggunakan 10-fold cross-validation
for fold, (train_index, test_index) in enumerate(zip(train_indices, test_indices)):
    # Membagi data menjadi data pelatihan dan data pengujian
    X_train_fold, y_train_fold = X.iloc[train_index], y.iloc[train_index]

    # Inisialisasi model Logistic Regression
    model_logreg = LogisticRegression(solver='liblinear', max_iter=1000, random_state=42)

    # Melatih model
    model_logreg.fit(X_train_fold, y_train_fold)

    # Simpan model yang sudah dilatih
    logreg_trained_models.append(model_logreg)

    # Menampilkan status pelatihan model
    print(f"Fold {fold + 1} - Logistic Regression model training complete")

Fold 1 - Logistic Regression model training complete
Fold 2 - Logistic Regression model training complete
Fold 3 - Logistic Regression model training complete
Fold 4 - Logistic Regression model training complete
Fold 5 - Logistic Regression model training complete
Fold 6 - Logistic Regression model training complete
Fold 7 - Logistic Regression model training complete
Fold 8 - Logistic Regression model training complete
Fold 9 - Logistic Regression model training complete
Fold 10 - Logistic Regression model training complete


### 4.2 Pengukuran Kinerja Model

In [ ]:
# Variabel untuk menyimpan prediksi dan probabilitas
y_preds = []
y_probs = []

# Loop untuk melakukan prediksi dengan Logistic Regression
for fold, (train_index, test_index) in enumerate(zip(train_indices, test_indices)):
    # Mengambil data pengujian
    X_test_fold = X.iloc[test_index]
    y_test_fold = y.iloc[test_index]

    # Mengambil model Logistic Regression yang sudah dilatih di fold ini
    model_logreg = logreg_trained_models[fold]

    # Melakukan prediksi pada data pengujian
    y_pred = model_logreg.predict(X_test_fold)
    y_preds.append(y_pred)

    # Mengambil probabilitas prediksi untuk semua kelas
    y_prob = model_logreg.predict_proba(X_test_fold)

    # Menyimpan probabilitas
    y_probs.append(y_prob)

    print(f"Fold {fold + 1} - Logistic Regression predictions complete")

Fold 1 - Logistic Regression predictions complete
Fold 2 - Logistic Regression predictions complete
Fold 3 - Logistic Regression predictions complete
Fold 4 - Logistic Regression predictions complete
Fold 5 - Logistic Regression predictions complete
Fold 6 - Logistic Regression predictions complete
Fold 7 - Logistic Regression predictions complete
Fold 8 - Logistic Regression predictions complete
Fold 9 - Logistic Regression predictions complete
Fold 10 - Logistic Regression predictions complete


### 4.3 Analisis Kinerja Model

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Variabel untuk menyimpan hasil kinerja
results = {
    'fold': [],
    'accuracy': [],
    'precision': [],
    'recall': [],
    'f1_score': [],
    'roc_auc': []
}

# Loop untuk menghitung kinerja model Logistic Regression
for fold, (train_index, test_index) in enumerate(zip(train_indices, test_indices)):
    # Mengambil data pengujian yang sesuai
    y_test_fold = y.iloc[test_index]
    y_pred = y_preds[fold]  # Prediksi dari Logistic Regression untuk fold ini
    y_prob = y_probs[fold]  # Probabilitas dari Logistic Regression untuk fold ini

    # Menghitung metrik kinerja
    accuracy = accuracy_score(y_test_fold, y_pred)
    precision = precision_score(y_test_fold, y_pred, average='macro', zero_division=0)
    recall = recall_score(y_test_fold, y_pred, average='macro', zero_division=0)
    f1 = f1_score(y_test_fold, y_pred, average='macro', zero_division=0)

    # Menghitung AUC ROC
    roc_auc = None
    num_classes = len(set(y_test_fold))  # Menghitung jumlah kelas

    if num_classes > 2:
        # Multi-class: ROC AUC menggunakan 'macro'
        try:
            roc_auc = roc_auc_score(y_test_fold, y_prob, multi_class='ovr', average='macro')
        except ValueError as e:
            print(f"Error calculating ROC AUC for fold {fold + 1}: {e}")
    elif num_classes == 2:
        # Binary classification: Ambil probabilitas untuk kelas positif
        try:
            roc_auc = roc_auc_score(y_test_fold, y_prob[:, 1])
        except ValueError as e:
            print(f"Error calculating ROC AUC for fold {fold + 1}: {e}")

    # Simpan hasil ke dalam dictionary
    results['fold'].append(fold + 1)
    results['accuracy'].append(accuracy)
    results['precision'].append(precision)
    results['recall'].append(recall)
    results['f1_score'].append(f1)
    results['roc_auc'].append(roc_auc)

    # Tampilkan hasil per fold
    if roc_auc is not None:
        print(f"Fold {fold + 1} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, "
              f"Recall: {recall:.4f}, F1 Score: {f1:.4f}, ROC AUC: {roc_auc:.4f}, "
              f"Number of Classes: {num_classes}")
    else:
        print(f"Fold {fold + 1} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, "
              f"Recall: {recall:.4f}, F1 Score: {f1:.4f}, ROC AUC: Not Applicable, "
              f"Number of Classes: {num_classes}")


Fold 1 - Accuracy: 0.9250, Precision: 0.4625, Recall: 0.5000, F1 Score: 0.4805, ROC AUC: 0.6007, Number of Classes: 2
Fold 2 - Accuracy: 0.9250, Precision: 0.4625, Recall: 0.5000, F1 Score: 0.4805, ROC AUC: 0.5968, Number of Classes: 2
Fold 3 - Accuracy: 0.9250, Precision: 0.4625, Recall: 0.5000, F1 Score: 0.4805, ROC AUC: 0.5517, Number of Classes: 2
Fold 4 - Accuracy: 0.9250, Precision: 0.4625, Recall: 0.5000, F1 Score: 0.4805, ROC AUC: 0.6014, Number of Classes: 2
Fold 5 - Accuracy: 0.9250, Precision: 0.4625, Recall: 0.5000, F1 Score: 0.4805, ROC AUC: 0.5697, Number of Classes: 2
Fold 6 - Accuracy: 0.9250, Precision: 0.4625, Recall: 0.5000, F1 Score: 0.4805, ROC AUC: 0.7110, Number of Classes: 2
Fold 7 - Accuracy: 0.9250, Precision: 0.4625, Recall: 0.5000, F1 Score: 0.4805, ROC AUC: 0.4901, Number of Classes: 2
Fold 8 - Accuracy: 0.9250, Precision: 0.4625, Recall: 0.5000, F1 Score: 0.4805, ROC AUC: 0.4746, Number of Classes: 2
Fold 9 - Accuracy: 0.9200, Precision: 0.4600, Recall: 0.

### 4.4 Training dan Validasi Model Terbaik

In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression

# 1. Mencari fold terbaik berdasarkan F1 Score
best_fold_index = np.argmax(results['f1_score'])
print(f"Fold terbaik berdasarkan F1 Score adalah Fold {results['fold'][best_fold_index]} dengan nilai F1 Score {results['f1_score'][best_fold_index]:.4f}")

# 2. Inisialisasi model Logistic Regression
best_model = logreg_trained_models[best_fold_index]  # Ambil model Logistic Regression dari fold terbaik

# 3. Latih ulang model menggunakan seluruh data (X dan y)
best_model = best_model.fit(X, y)  # Latih ulang model dengan semua data


Fold terbaik berdasarkan F1 Score adalah Fold 1 dengan nilai F1 Score 0.4805


### 4.5 Pengukuran Model Terbaik

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# 1. Prediksi label dan probabilitas untuk data baru
y_pred_new = best_model.predict(X)  # Prediksi label
y_probs_new = best_model.predict_proba(X)  # Prediksi probabilitas

# 2. Evaluasi kinerja pada data baru
accuracy_new = accuracy_score(y, y_pred_new)
precision_new = precision_score(y, y_pred_new, average='macro', zero_division=0)
recall_new = recall_score(y, y_pred_new, average='macro', zero_division=0)
f1_new = f1_score(y, y_pred_new, average='macro', zero_division=0)

# 3. Hitung ROC AUC (dengan penanganan error jika terjadi)
try:
    roc_auc_new = roc_auc_score(y, y_probs_new, multi_class='ovr')
except ValueError as e:
    roc_auc_new = None
    print(f"Error calculating ROC AUC on new data: {e}")

# 4. Cetak hasil evaluasi
print("Evaluasi pada data baru:")
print(f"Accuracy: {accuracy_new:.4f}")
print(f"Precision: {precision_new:.4f}")
print(f"Recall: {recall_new:.4f}")
print(f"F1 Score: {f1_new:.4f}")
if roc_auc_new is not None:
    print(f"ROC AUC: {roc_auc_new:.4f}")
else:
    print("ROC AUC: Not Applicable")


Error calculating ROC AUC on new data: y should be a 1d array, got an array of shape (2000, 2) instead.
Evaluasi pada data baru:
Accuracy: 0.9245
Precision: 0.9622
Recall: 0.5033
F1 Score: 0.4869
ROC AUC: Not Applicable


# 5 - Eksperimen dengan Model Klasifikasi SVM

### Tujuan
Mengimplementasikan metode klasifikasi SVM dan melakukan analisis dari hasil eksperimen.

### Langkah-Langkah
1. **Eksperimen dengan Kombinasi Kernel dan Parameter C**
   - Lakukan eksperimen untuk memperoleh model klasifikasi SVM terbaik menggunakan kombinasi dua jenis kernel: linear dan radial basis function (RBF), serta tiga parameter C yang dapat Anda tentukan sendiri dalam eksperimen.
   - Total akan terdapat enam kombinasi nilai parameter kernel dan C.

2. **Validasi dengan 10-Fold Cross Validation**
   - Untuk setiap kombinasi nilai seperti disebutkan dalam langkah pertama, jalankan model klasifikasi SVM menggunakan *10-fold cross validation*.
   - Setiap kombinasi nilai parameter SVM akan memberikan 10 hasil validasi dari model.

3. **Pengukuran Kinerja Model**
   - Hitung akurasi, presisi, *recall*, F1-score, dan AUC ROC dari setiap data validasi pada langkah ke-2.

4. **Analisis Kinerja Model**
   - Lakukan analisis dari 10 hasil kinerja model SVM untuk satu kombinasi parameter SVM yang sedang diuji berdasarkan hasil pada langkah ke-3.

5. **Penentuan Model Terbaik**
   - Berdasarkan analisis pada langkah ke-4, tentukan kombinasi parameter yang Anda anggap menghasilkan model terbaik.
   - Jalankan model SVM pada keseluruhan data pelatihan menggunakan kombinasi parameter tersebut dan hitung kinerja model (akurasi, presisi, *recall*, F1-score, dan AUC ROC).

6. **Ulangi untuk Kombinasi Nilai Berikutnya**
   - Ulangi langkah ke-2 hingga ke-5 untuk kombinasi nilai parameter SVM berikutnya.

7. **Analisis Hasil Kinerja Model**
   - Lakukan analisis dari hasil kinerja kelima model SVM yang diperoleh pada langkah ke-6.

8. **Prediksi dengan Model Terbaik**
   - Gunakan model SVM terbaik untuk memprediksi semua record dari data tes.
   - Hitung akurasi, presisi, *recall*, F1-score, dan AUC ROC dari model terbaik pada data tes (data yang tidak digunakan dalam membangun model terbaik).


### 5.1 Eksperimen dengan Kombinasi Kernel dan Parameter C

In [ ]:
from itertools import product  # Untuk membuat kombinasi parameter

# Definisikan kernel dan nilai C untuk eksperimen
kernels = ['linear', 'rbf']
C_values = [0.01, 0.1, 1, 10, 100]

# Buat semua kombinasi parameter
combinations = list(product(kernels, C_values))

# Tampilkan kombinasi parameter
print("Kombinasi Kernel dan C yang akan diuji:")
for kernel, C in combinations:
    print(f"Kernel: {kernel}, C: {C}")

Kombinasi Kernel dan C yang akan diuji:
Kernel: linear, C: 0.01
Kernel: linear, C: 0.1
Kernel: linear, C: 1
Kernel: linear, C: 10
Kernel: linear, C: 100
Kernel: rbf, C: 0.01
Kernel: rbf, C: 0.1
Kernel: rbf, C: 1
Kernel: rbf, C: 10
Kernel: rbf, C: 100


### 5.2 Validasi dengan 10-Fold Cross Validation

In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

# Standarisasi data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Variabel untuk menyimpan model yang dilatih per kombinasi
svm_trained_models = {}

# Loop untuk setiap kombinasi kernel dan C
for kernel, C in combinations:
    print(f"\nTraining SVM dengan Kernel={kernel} dan C={C}")
    models_per_combination = []

    # Loop untuk setiap fold dengan indeks yang sudah disimpan
    for fold, (train_index, test_index) in enumerate(zip(train_indices, test_indices), start=1):
        # Ambil data train dan validasi berdasarkan indeks yang ditetapkan
        X_train_fold, y_train_fold = X_scaled[train_index], y.iloc[train_index]
        X_val_fold, y_val_fold = X_scaled[test_index], y.iloc[test_index]

        # Inisialisasi dan latih model SVM
        model = SVC(kernel=kernel, C=C, probability=True, random_state=42)
        model.fit(X_train_fold, y_train_fold)

        # Simpan model yang sudah dilatih
        models_per_combination.append(model)

        print(f"Fold {fold} - Training completed")

    # Simpan semua model untuk kombinasi kernel dan C saat ini
    svm_trained_models[(kernel, C)] = models_per_combination

print("\nTraining selesai untuk semua kombinasi.")


Training SVM dengan Kernel=linear dan C=0.01
Fold 1 - Training completed
Fold 2 - Training completed
Fold 3 - Training completed
Fold 4 - Training completed
Fold 5 - Training completed
Fold 6 - Training completed
Fold 7 - Training completed
Fold 8 - Training completed
Fold 9 - Training completed
Fold 10 - Training completed

Training SVM dengan Kernel=linear dan C=0.1
Fold 1 - Training completed
Fold 2 - Training completed
Fold 3 - Training completed
Fold 4 - Training completed
Fold 5 - Training completed
Fold 6 - Training completed
Fold 7 - Training completed
Fold 8 - Training completed
Fold 9 - Training completed
Fold 10 - Training completed

Training SVM dengan Kernel=linear dan C=1
Fold 1 - Training completed
Fold 2 - Training completed
Fold 3 - Training completed
Fold 4 - Training completed
Fold 5 - Training completed
Fold 6 - Training completed
Fold 7 - Training completed
Fold 8 - Training completed
Fold 9 - Training completed
Fold 10 - Training completed

Training SVM dengan Ke

### 5.3 Pengukuran Kinerja Model

In [ ]:
# Variabel untuk menyimpan prediksi dan probabilitas per kombinasi kernel dan C
y_preds = {}
y_probs = {}

# Testing dengan model yang sudah dilatih
for (kernel, C), models in svm_trained_models.items():
    print(f"\nGenerating predictions for Kernel={kernel} and C={C}")

    # Variabel untuk menyimpan hasil per fold
    fold_y_preds = []
    fold_y_probs = []

    # Loop melalui setiap fold menggunakan indeks yang disimpan
    for fold, (train_index, test_index) in enumerate(zip(train_indices, test_indices), start=1):
        # Ambil model untuk fold ini
        model = models[fold - 1]

        # Ambil data validasi untuk fold ini berdasarkan indeks
        X_test_fold, y_test_fold = X_scaled[test_index], y.iloc[test_index]

        # Lakukan prediksi dan hitung probabilitas
        y_pred = model.predict(X_test_fold)
        y_prob = model.predict_proba(X_test_fold)

        # Simpan hasil prediksi dan probabilitas
        fold_y_preds.append(y_pred)
        fold_y_probs.append(y_prob)

        print(f"Fold {fold} - Predictions completed")

    # Simpan hasil prediksi dan probabilitas untuk kombinasi kernel dan C saat ini
    y_preds[(kernel, C)] = fold_y_preds
    y_probs[(kernel, C)] = fold_y_probs


Generating predictions for Kernel=linear and C=0.01
Fold 1 - Predictions completed
Fold 2 - Predictions completed
Fold 3 - Predictions completed
Fold 4 - Predictions completed
Fold 5 - Predictions completed
Fold 6 - Predictions completed
Fold 7 - Predictions completed
Fold 8 - Predictions completed
Fold 9 - Predictions completed
Fold 10 - Predictions completed

Generating predictions for Kernel=linear and C=0.1
Fold 1 - Predictions completed
Fold 2 - Predictions completed
Fold 3 - Predictions completed
Fold 4 - Predictions completed
Fold 5 - Predictions completed
Fold 6 - Predictions completed
Fold 7 - Predictions completed
Fold 8 - Predictions completed
Fold 9 - Predictions completed
Fold 10 - Predictions completed

Generating predictions for Kernel=linear and C=1
Fold 1 - Predictions completed
Fold 2 - Predictions completed
Fold 3 - Predictions completed
Fold 4 - Predictions completed
Fold 5 - Predictions completed
Fold 6 - Predictions completed
Fold 7 - Predictions completed
Fold 8

### 5.4 Analisis Kinerja Model

In [ ]:
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score
)
from sklearn.preprocessing import label_binarize
import numpy as np
import pandas as pd

# Variabel untuk menyimpan hasil metrik
results = {
    'kernel': [], 'C': [], 'fold': [],
    'accuracy': [], 'precision': [], 'recall': [],
    'f1_score': [], 'roc_auc': []
}

# Loop untuk menghitung metrik kinerja dari hasil prediksi
for (kernel, C), fold_y_preds in y_preds.items():
    print(f"\nEvaluating performance for Kernel={kernel} and C={C}")

    # Loop melalui setiap fold
    for fold, (y_pred, y_prob) in enumerate(zip(fold_y_preds, y_probs[(kernel, C)]), start=1):
        # Ambil data validasi untuk fold ini menggunakan indeks yang disimpan
        _, test_index = train_indices[fold - 1], test_indices[fold - 1]
        y_val_fold = y.iloc[test_index]

        # Hitung metrik evaluasi
        accuracy = accuracy_score(y_val_fold, y_pred)
        precision = precision_score(y_val_fold, y_pred, average='macro', zero_division=0)
        recall = recall_score(y_val_fold, y_pred, average='macro', zero_division=0)
        f1 = f1_score(y_val_fold, y_pred, average='macro', zero_division=0)

        # Hitung ROC AUC
        # Jika y_prob 2D:
        if y_prob.ndim > 1:
            if y_prob.shape[1] == 2:
                # Klasifikasi biner: ambil probabilitas kelas positif (label=1)
                roc_auc = roc_auc_score(y_val_fold, y_prob[:, 1])
            else:
                # Multiclass: binarisasi label untuk ROC AUC multiclass
                classes = np.unique(y)
                y_true_bin = label_binarize(y_val_fold, classes=classes)
                roc_auc = roc_auc_score(
                    y_true_bin,
                    y_prob,
                    multi_class='ovr',
                    average='macro'
                )
        else:
            # Jika y_prob sudah 1D (misal hanya skor untuk kelas positif)
            roc_auc = roc_auc_score(y_val_fold, y_prob)

        # Simpan hasil metrik
        results['kernel'].append(kernel)
        results['C'].append(C)
        results['fold'].append(fold)
        results['accuracy'].append(accuracy)
        results['precision'].append(precision)
        results['recall'].append(recall)
        results['f1_score'].append(f1)
        results['roc_auc'].append(roc_auc)

        # Tampilkan hasil per fold
        print(
            f"Fold {fold} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, "
            f"Recall: {recall:.4f}, F1 Score: {f1:.4f}, ROC AUC: {roc_auc:.4f}"
        )

# Setelah looping semua kombinasi dan fold:
results_df = pd.DataFrame(results)
print("\nAll metrics collected in DataFrame:")
print(results_df.head())



Evaluating performance for Kernel=linear and C=0.01
Fold 1 - Accuracy: 0.9250, Precision: 0.4625, Recall: 0.5000, F1 Score: 0.4805, ROC AUC: 0.6850
Fold 2 - Accuracy: 0.9250, Precision: 0.4625, Recall: 0.5000, F1 Score: 0.4805, ROC AUC: 0.5964
Fold 3 - Accuracy: 0.9250, Precision: 0.4625, Recall: 0.5000, F1 Score: 0.4805, ROC AUC: 0.5279
Fold 4 - Accuracy: 0.9250, Precision: 0.4625, Recall: 0.5000, F1 Score: 0.4805, ROC AUC: 0.5686
Fold 5 - Accuracy: 0.9250, Precision: 0.4625, Recall: 0.5000, F1 Score: 0.4805, ROC AUC: 0.5470
Fold 6 - Accuracy: 0.9250, Precision: 0.4625, Recall: 0.5000, F1 Score: 0.4805, ROC AUC: 0.5348
Fold 7 - Accuracy: 0.9250, Precision: 0.4625, Recall: 0.5000, F1 Score: 0.4805, ROC AUC: 0.6782
Fold 8 - Accuracy: 0.9250, Precision: 0.4625, Recall: 0.5000, F1 Score: 0.4805, ROC AUC: 0.3636
Fold 9 - Accuracy: 0.9200, Precision: 0.4600, Recall: 0.5000, F1 Score: 0.4792, ROC AUC: 0.7211
Fold 10 - Accuracy: 0.9200, Precision: 0.4600, Recall: 0.5000, F1 Score: 0.4792, RO

### 5.5 Training Model Terbaik

In [ ]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

# 1. Temukan kombinasi terbaik berdasarkan rata-rata accuracy
mean_accuracy = results_df.groupby(['kernel', 'C'])['accuracy'].mean().reset_index()
best_combination = mean_accuracy.loc[mean_accuracy['accuracy'].idxmax()]

# 2. Filter data berdasarkan kombinasi terbaik
best_kernel = best_combination['kernel']
best_C = best_combination['C']

# Ambil semua hasil fold untuk kombinasi terbaik
best_model_folds = results_df[(results_df['kernel'] == best_kernel) &
                              (results_df['C'] == best_C)]

# 3. Temukan fold terbaik berdasarkan accuracy tertinggi
best_fold = best_model_folds.loc[best_model_folds['accuracy'].idxmax()]
best_fold_number = best_fold['fold']  # Dapatkan nomor fold terbaik

# 4. Tampilkan hasil analisis
print("Kombinasi Terbaik Berdasarkan Accuracy:")
print(f"Kernel: {best_combination['kernel']}, C: {best_combination['C']}, "
      f"Mean Accuracy: {best_combination['accuracy']:.4f}")

print(f"\nFold Terbaik dari Kombinasi Terbaik:")
print(f"Fold: {best_fold['fold']}, Accuracy: {best_fold['accuracy']:.4f}, "
      f"Precision: {best_fold['precision']:.4f}, Recall: {best_fold['recall']:.4f}, "
      f"F1 Score: {best_fold['f1_score']:.4f}, ROC AUC: {best_fold['roc_auc']:.4f}")

# 5. Ambil model terbaik berdasarkan kombinasi dan fold terbaik
best_model = svm_trained_models[(best_kernel, best_C)][best_fold_number - 1]

# 6. Standarisasi ulang data untuk training model terbaik pada seluruh data train
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 7. Latih model terbaik menggunakan seluruh data train
print("\nTraining ulang model terbaik pada seluruh data train...")
best_model.fit(X_scaled, y)

print("Training selesai. Model siap digunakan.")

Kombinasi Terbaik Berdasarkan Accuracy:
Kernel: linear, C: 0.01, Mean Accuracy: 0.9240

Fold Terbaik dari Kombinasi Terbaik:
Fold: 1, Accuracy: 0.9250, Precision: 0.4625, Recall: 0.5000, F1 Score: 0.4805, ROC AUC: 0.6850

Training ulang model terbaik pada seluruh data train...
Training selesai. Model siap digunakan.


### 5.6 Validasi dan Pengukuran Kinerja Model Terbaik

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np

# 1. Prediksi label untuk seluruh data menggunakan model terbaik
print("\nMelakukan prediksi dengan model SVM terbaik...")
y_pred_new = best_model.predict(X_scaled)  # Gunakan data X yang sudah distandarisasi
y_probs_new = best_model.predict_proba(X_scaled)

# 2. Evaluasi kinerja pada seluruh data
accuracy_new = accuracy_score(y, y_pred_new)
precision_new = precision_score(y, y_pred_new, average='macro', zero_division=0)
recall_new = recall_score(y, y_pred_new, average='macro', zero_division=0)
f1_new = f1_score(y, y_pred_new, average='macro', zero_division=0)

# 3. Hitung ROC AUC
try:
    if len(np.unique(y)) > 2:  # Multiclass
        roc_auc_new = roc_auc_score(y, y_probs_new, multi_class='ovr', average='macro')
    else:  # Binary classification
        roc_auc_new = roc_auc_score(y, y_probs_new[:, 1])
except ValueError as e:
    roc_auc_new = None
    print(f"Error calculating ROC AUC on new data: {e}")

# 4. Cetak hasil evaluasi
print("\nEvaluasi Kinerja Model Terbaik pada Data:")
print(f"Accuracy: {accuracy_new:.4f}")
print(f"Precision: {precision_new:.4f}")
print(f"Recall: {recall_new:.4f}")
print(f"F1 Score: {f1_new:.4f}")

if roc_auc_new is not None:
    print(f"ROC AUC: {roc_auc_new:.4f}")
else:
    print("ROC AUC: Not Applicable")


Melakukan prediksi dengan model SVM terbaik...

Evaluasi Kinerja Model Terbaik pada Data:
Accuracy: 0.9245
Precision: 0.9622
Recall: 0.5033
F1 Score: 0.4869
ROC AUC: 0.6241


#6 - Eksperimen dengan Model Klasifikasi Decision Tree (DT)

### Tujuan
Mengimplementasikan metode klasifikasi DT dan melakukan analisis dari hasil eksperimen.

### Langkah-Langkah
1. **Eksperimen dengan Kombinasi Nilai Parameter**
   - Lakukan eksperimen untuk memperoleh model klasifikasi DT terbaik menggunakan kombinasi nilai parameter berikut (nilai untuk setiap parameter dapat ditentukan sendiri dalam eksperimen). Gunakan nilai default untuk parameter DT lainnya:
     - **max_depth** (2 nilai): Tingkat kedalaman maksimum dari pohon.
     - **min_samples_split** (2 nilai): Jumlah minimum sampel yang dibutuhkan untuk membagi sebuah simpul (node).
     - **min_samples_leaf** (2 nilai): Jumlah minimum sampel yang diperlukan untuk menjadi daun (leaf).
     - **max_features** (2 nilai): Jumlah maksimum fitur yang dipertimbangkan untuk setiap split.
     - **max_leaf_nodes** (2 nilai): Jumlah maksimum daun (leaf nodes) dalam pohon.
     - **criterion** (gini dan entropy): Kualitas dari split yang dilakukan pada setiap simpul.
   - Kombinasi nilai parameter ini akan memerlukan eksperimen untuk 64 kombinasi nilai parameter yang berbeda.

2. **Validasi dengan 10-Fold Cross Validation**
   - Untuk setiap kombinasi nilai seperti disebutkan dalam langkah pertama, jalankan model klasifikasi Decision Tree menggunakan *10-fold cross validation*.
   - Setiap kombinasi nilai parameter DT akan memberikan 10 hasil validasi dari model.

3. **Pengukuran Kinerja Model**
   - Hitung akurasi, presisi, *recall*, F1-score, dan AUC ROC dari setiap data validasi pada langkah ke-2.

4. **Analisis Kinerja Model**
   - Lakukan analisis dari lima hasil kinerja model DT untuk satu kombinasi parameter DT yang sedang diuji berdasarkan hasil pada langkah ke-3.

5. **Penentuan Model Terbaik**
   - Berdasarkan analisis pada langkah ke-4, tentukan kombinasi parameter yang Anda anggap menghasilkan model terbaik.
   - Jalankan model DT pada keseluruhan data pelatihan menggunakan kombinasi parameter tersebut dan hitung kinerja model (akurasi, presisi, *recall*, F1-score, dan AUC ROC).

6. **Ulangi untuk Kombinasi Nilai Berikutnya**
   - Ulangi langkah ke-2 hingga ke-5 untuk kombinasi nilai parameter DT berikutnya.

7. **Analisis Hasil Kinerja Model**
   - Lakukan analisis dari hasil kinerja kelima model DT yang diperoleh pada langkah ke-6.

8. **Prediksi dengan Model Terbaik**
   - Gunakan model DT terbaik untuk memprediksi semua record dari data tes.
   - Hitung akurasi, presisi, *recall*, F1-score, dan AUC ROC dari model terbaik pada data tes (data yang tidak digunakan dalam membangun model terbaik).


### 6.1 Eksperimen dengan Kombinasi Nilai Parameter

In [ ]:
from itertools import product  # Untuk membuat kombinasi parameter

# Definisikan nilai-nilai parameter
max_depth = [5, 10]
min_samples_split = [2, 5]
min_samples_leaf = [1, 2]
max_features = [None, 'sqrt']
max_leaf_nodes = [None, 10]
criteria = ['gini', 'entropy']

# Buat semua kombinasi parameter
combinations = list(product(
    max_depth, min_samples_split, min_samples_leaf,
    max_features, max_leaf_nodes, criteria
))

# Tampilkan kombinasi parameter
print("Kombinasi Parameter untuk Eksperimen:")
for params in combinations:
    print(f"max_depth: {params[0]}, min_samples_split: {params[1]}, "
          f"min_samples_leaf: {params[2]}, max_features: {params[3]}, "
          f"max_leaf_nodes: {params[4]}, criterion: {params[5]}")


Kombinasi Parameter untuk Eksperimen:
max_depth: 5, min_samples_split: 2, min_samples_leaf: 1, max_features: None, max_leaf_nodes: None, criterion: gini
max_depth: 5, min_samples_split: 2, min_samples_leaf: 1, max_features: None, max_leaf_nodes: None, criterion: entropy
max_depth: 5, min_samples_split: 2, min_samples_leaf: 1, max_features: None, max_leaf_nodes: 10, criterion: gini
max_depth: 5, min_samples_split: 2, min_samples_leaf: 1, max_features: None, max_leaf_nodes: 10, criterion: entropy
max_depth: 5, min_samples_split: 2, min_samples_leaf: 1, max_features: sqrt, max_leaf_nodes: None, criterion: gini
max_depth: 5, min_samples_split: 2, min_samples_leaf: 1, max_features: sqrt, max_leaf_nodes: None, criterion: entropy
max_depth: 5, min_samples_split: 2, min_samples_leaf: 1, max_features: sqrt, max_leaf_nodes: 10, criterion: gini
max_depth: 5, min_samples_split: 2, min_samples_leaf: 1, max_features: sqrt, max_leaf_nodes: 10, criterion: entropy
max_depth: 5, min_samples_split: 2, mi

### 6.2 Validasi dengan 10-Fold Cross Validation

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler

# Standarisasi data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Variabel untuk menyimpan model per kombinasi parameter
dt_trained_models = {}

# Loop untuk setiap kombinasi parameter
for params in combinations:
    max_d, min_split, min_leaf, max_feat, max_leaf, crit = params
    print(f"\nTraining Decision Tree dengan Parameter: {params}")

    models_per_combination = []

    # Loop melalui setiap fold dengan indeks yang disimpan
    for fold, (train_index, test_index) in enumerate(zip(train_indices, test_indices), start=1):
        try:
            # Validasi agar indeks sesuai dengan ukuran data
            if max(train_index) >= len(X_scaled) or max(test_index) >= len(X_scaled):
                raise IndexError("Indeks melebihi batas ukuran data.")

            # Ambil data training dan validasi berdasarkan indeks
            X_train_fold, y_train_fold = X_scaled[train_index], y.iloc[train_index].values
            X_val_fold, y_val_fold = X_scaled[test_index], y.iloc[test_index].values

            # Inisialisasi model Decision Tree dengan parameter saat ini
            model = DecisionTreeClassifier(
                max_depth=max_d, min_samples_split=min_split,
                min_samples_leaf=min_leaf, max_features=max_feat,
                max_leaf_nodes=max_leaf, criterion=crit, random_state=42
            )

            # Latih model pada data training untuk fold saat ini
            model.fit(X_train_fold, y_train_fold)

            # Simpan model yang sudah dilatih
            models_per_combination.append(model)

            print(f"Fold {fold} - Training completed")

        except IndexError as e:
            print(f"Error on fold {fold}: {e}")
            continue  # Lewati fold yang memiliki masalah indeks

    # Simpan semua model untuk kombinasi parameter saat ini
    dt_trained_models[params] = models_per_combination

print("\nTraining selesai untuk semua kombinasi.")



Training Decision Tree dengan Parameter: (5, 2, 1, None, None, 'gini')
Fold 1 - Training completed
Fold 2 - Training completed
Fold 3 - Training completed
Fold 4 - Training completed
Fold 5 - Training completed
Fold 6 - Training completed
Fold 7 - Training completed
Fold 8 - Training completed
Fold 9 - Training completed
Fold 10 - Training completed

Training Decision Tree dengan Parameter: (5, 2, 1, None, None, 'entropy')
Fold 1 - Training completed
Fold 2 - Training completed
Fold 3 - Training completed
Fold 4 - Training completed
Fold 5 - Training completed
Fold 6 - Training completed
Fold 7 - Training completed
Fold 8 - Training completed
Fold 9 - Training completed
Fold 10 - Training completed

Training Decision Tree dengan Parameter: (5, 2, 1, None, 10, 'gini')
Fold 1 - Training completed
Fold 2 - Training completed
Fold 3 - Training completed
Fold 4 - Training completed
Fold 5 - Training completed
Fold 6 - Training completed
Fold 7 - Training completed
Fold 8 - Training complet

### 6.3 Pengukuran Kinerja Model

In [ ]:
# Variabel untuk menyimpan prediksi dan probabilitas per kombinasi parameter
y_preds = {}
y_probs = {}

# Loop untuk setiap kombinasi parameter
for params, models in dt_trained_models.items():
    print(f"\nGenerating predictions for Parameter: {params}")

    # Variabel untuk menyimpan hasil per fold
    fold_y_preds = []
    fold_y_probs = []

    # Loop melalui setiap fold menggunakan indeks yang disimpan
    for fold, (train_index, test_index) in enumerate(zip(train_indices, test_indices), start=1):
        # Ambil model untuk fold ini
        model = models[fold - 1]

        # Ambil data validasi berdasarkan indeks
        X_test_fold, y_test_fold = X_scaled[test_index], y.iloc[test_index]

        # Lakukan prediksi dan hitung probabilitas
        y_pred = model.predict(X_test_fold)
        y_prob = model.predict_proba(X_test_fold)

        # Simpan hasil prediksi dan probabilitas
        fold_y_preds.append(y_pred)
        fold_y_probs.append(y_prob)

        print(f"Fold {fold} - Predictions completed")

    # Simpan hasil prediksi dan probabilitas untuk kombinasi parameter saat ini
    y_preds[params] = fold_y_preds
    y_probs[params] = fold_y_probs

print("\nPredictions selesai untuk semua kombinasi.")


Generating predictions for Parameter: (5, 2, 1, None, None, 'gini')
Fold 1 - Predictions completed
Fold 2 - Predictions completed
Fold 3 - Predictions completed
Fold 4 - Predictions completed
Fold 5 - Predictions completed
Fold 6 - Predictions completed
Fold 7 - Predictions completed
Fold 8 - Predictions completed
Fold 9 - Predictions completed
Fold 10 - Predictions completed

Generating predictions for Parameter: (5, 2, 1, None, None, 'entropy')
Fold 1 - Predictions completed
Fold 2 - Predictions completed
Fold 3 - Predictions completed
Fold 4 - Predictions completed
Fold 5 - Predictions completed
Fold 6 - Predictions completed
Fold 7 - Predictions completed
Fold 8 - Predictions completed
Fold 9 - Predictions completed
Fold 10 - Predictions completed

Generating predictions for Parameter: (5, 2, 1, None, 10, 'gini')
Fold 1 - Predictions completed
Fold 2 - Predictions completed
Fold 3 - Predictions completed
Fold 4 - Predictions completed
Fold 5 - Predictions completed
Fold 6 - Predic

### 6.4 Analisis Kinerja Model

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import pandas as pd
import numpy as np

# Variabel untuk menyimpan hasil metrik evaluasi
results = {
    'params': [], 'fold': [],
    'accuracy': [], 'precision': [], 'recall': [],
    'f1_score': [], 'roc_auc': []
}

# Loop untuk setiap kombinasi parameter
for params, fold_y_preds in y_preds.items():
    print(f"\nEvaluating performance for Parameter: {params}")

    # Loop melalui setiap fold
    for fold, (y_pred, y_prob) in enumerate(zip(fold_y_preds, y_probs[params]), start=1):
        # Ambil data validasi untuk fold ini
        _, test_index = train_indices[fold - 1], test_indices[fold - 1]
        y_test_fold = y.iloc[test_index]

        # Hitung metrik evaluasi
        accuracy = accuracy_score(y_test_fold, y_pred)
        precision = precision_score(y_test_fold, y_pred, average='macro', zero_division=0)
        recall = recall_score(y_test_fold, y_pred, average='macro', zero_division=0)
        f1 = f1_score(y_test_fold, y_pred, average='macro', zero_division=0)

        # Hitung ROC AUC
        try:
            if len(np.unique(y_test_fold)) > 2:
                roc_auc = roc_auc_score(y_test_fold, y_prob, multi_class='ovr', average='macro')
            else:
                roc_auc = roc_auc_score(y_test_fold, y_prob[:, 1])
        except ValueError as e:
            roc_auc = None
            print(f"ROC AUC Error on fold {fold}: {e}")

        # Simpan hasil metrik
        results['params'].append(params)
        results['fold'].append(fold)
        results['accuracy'].append(accuracy)
        results['precision'].append(precision)
        results['recall'].append(recall)
        results['f1_score'].append(f1)
        results['roc_auc'].append(roc_auc)

        # Tampilkan hasil metrik untuk setiap fold
        roc_auc_display = f"{roc_auc:.4f}" if roc_auc is not None else "Not Applicable"
        print(f"Fold {fold} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, "
              f"Recall: {recall:.4f}, F1 Score: {f1:.4f}, ROC AUC: {roc_auc_display}")

# Konversi hasil metrik menjadi DataFrame untuk analisis lebih lanjut
results_df = pd.DataFrame(results)

print("\nMetrik evaluasi selesai dihitung untuk semua kombinasi dan fold.")



Evaluating performance for Parameter: (5, 2, 1, None, None, 'gini')
Fold 1 - Accuracy: 0.9200, Precision: 0.6311, Recall: 0.5279, F1 Score: 0.5346, ROC AUC: 0.6099
Fold 2 - Accuracy: 0.9000, Precision: 0.4615, Recall: 0.4865, F1 Score: 0.4737, ROC AUC: 0.5656
Fold 3 - Accuracy: 0.9250, Precision: 0.7146, Recall: 0.5306, F1 Score: 0.5392, ROC AUC: 0.7384
Fold 4 - Accuracy: 0.9200, Precision: 0.4623, Recall: 0.4973, F1 Score: 0.4792, ROC AUC: 0.6495
Fold 5 - Accuracy: 0.9200, Precision: 0.4623, Recall: 0.4973, F1 Score: 0.4792, ROC AUC: 0.5935
Fold 6 - Accuracy: 0.9250, Precision: 0.4625, Recall: 0.5000, F1 Score: 0.4805, ROC AUC: 0.6178
Fold 7 - Accuracy: 0.9100, Precision: 0.6353, Recall: 0.5838, F1 Score: 0.6011, ROC AUC: 0.5281
Fold 8 - Accuracy: 0.9250, Precision: 0.4625, Recall: 0.5000, F1 Score: 0.4805, ROC AUC: 0.6535
Fold 9 - Accuracy: 0.9200, Precision: 0.4600, Recall: 0.5000, F1 Score: 0.4792, ROC AUC: 0.7306
Fold 10 - Accuracy: 0.8750, Precision: 0.4581, Recall: 0.4755, F1 S

### 6.5 Penentuan Model Terbaik

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# 1. Temukan kombinasi terbaik berdasarkan rata-rata accuracy
mean_accuracy = results_df.groupby('params')['accuracy'].mean().reset_index()
best_combination = mean_accuracy.loc[mean_accuracy['accuracy'].idxmax()]

# 2. Filter data berdasarkan kombinasi terbaik
best_params = best_combination['params']

# Ambil semua hasil fold untuk kombinasi terbaik
best_model_folds = results_df[results_df['params'] == best_params]

# 3. Temukan fold terbaik berdasarkan accuracy tertinggi
best_fold = best_model_folds.loc[best_model_folds['accuracy'].idxmax()]
best_fold_number = best_fold['fold']  # Dapatkan nomor fold terbaik

# 4. Tampilkan hasil analisis
print("Kombinasi Terbaik Berdasarkan Accuracy:")
print(f"Parameters: {best_params}")
print(f"Mean Accuracy: {best_combination['accuracy']:.4f}")

print(f"\nFold Terbaik dari Kombinasi Terbaik:")
print(f"Fold: {best_fold['fold']}, Accuracy: {best_fold['accuracy']:.4f}, "
      f"Precision: {best_fold['precision']:.4f}, Recall: {best_fold['recall']:.4f}, "
      f"F1 Score: {best_fold['f1_score']:.4f}, ROC AUC: {best_fold['roc_auc']:.4f}")

# 5. Ambil model terbaik berdasarkan kombinasi dan fold terbaik
print(f"\nFold terbaik berada pada Fold {best_fold_number} untuk kombinasi {best_params}.")

# Pastikan model terbaik tersedia dalam dt_trained_models
best_model = dt_trained_models[best_params][best_fold_number - 1]

# 6. Standarisasi ulang data untuk training model terbaik pada seluruh data train
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 7. Latih model terbaik menggunakan seluruh data train
print("\nTraining ulang model terbaik pada seluruh data train...")
best_model.fit(X_scaled, y)

print("Training selesai. Model siap digunakan.")


Kombinasi Terbaik Berdasarkan Accuracy:
Parameters: (5, 2, 1, 'sqrt', 10, 'entropy')
Mean Accuracy: 0.9235

Fold Terbaik dari Kombinasi Terbaik:
Fold: 1, Accuracy: 0.9250, Precision: 0.4625, Recall: 0.5000, F1 Score: 0.4805, ROC AUC: 0.7499

Fold terbaik berada pada Fold 1 untuk kombinasi (5, 2, 1, 'sqrt', 10, 'entropy').

Training ulang model terbaik pada seluruh data train...
Training selesai. Model siap digunakan.


### 6.6 Prediksi dengan Model Terbaik

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np

# 1. Prediksi label untuk seluruh data menggunakan model Decision Tree terbaik
print("\nMelakukan prediksi dengan model Decision Tree terbaik...")
y_pred_new = best_model.predict(X_scaled)  # Gunakan data X yang sudah distandarisasi
y_probs_new = best_model.predict_proba(X_scaled)

# 2. Evaluasi kinerja pada seluruh data
accuracy_new = accuracy_score(y, y_pred_new)
precision_new = precision_score(y, y_pred_new, average='macro', zero_division=0)
recall_new = recall_score(y, y_pred_new, average='macro', zero_division=0)
f1_new = f1_score(y, y_pred_new, average='macro', zero_division=0)

# 3. Hitung ROC AUC
try:
    if len(np.unique(y)) > 2:  # Multiclass
        roc_auc_new = roc_auc_score(y, y_probs_new, multi_class='ovr', average='macro')
    else:  # Binary classification
        roc_auc_new = roc_auc_score(y, y_probs_new[:, 1])
except ValueError as e:
    roc_auc_new = None
    print(f"Error calculating ROC AUC on new data: {e}")

# 4. Cetak hasil evaluasi
print("\nEvaluasi Kinerja Model Terbaik pada Data:")
print(f"Accuracy: {accuracy_new:.4f}")
print(f"Precision: {precision_new:.4f}")
print(f"Recall: {recall_new:.4f}")
print(f"F1 Score: {f1_new:.4f}")

if roc_auc_new is not None:
    print(f"ROC AUC: {roc_auc_new:.4f}")
else:
    print("ROC AUC: Not Applicable")



Melakukan prediksi dengan model Decision Tree terbaik...

Evaluasi Kinerja Model Terbaik pada Data:
Accuracy: 0.9240
Precision: 0.4620
Recall: 0.5000
F1 Score: 0.4802
ROC AUC: 0.7507


# 7 - Eksperimen dengan Model Back Propagation Neural Network (BPNN)

### Tujuan
Mengimplementasikan metode klasifikasi BPNN dan melakukan analisis dari hasil eksperimen.

### Langkah-Langkah
1. **Eksperimen dengan Kombinasi Nilai Parameter**
   - Lakukan eksperimen untuk memperoleh model klasifikasi BPNN terbaik menggunakan kombinasi nilai parameter berikut (nilai untuk setiap parameter dapat ditentukan sendiri dalam eksperimen). Gunakan nilai default untuk parameter BPNN lainnya:
     - **Jumlah Hidden Layers** (1 dan 2): Jumlah lapisan tersembunyi antara lapisan input dan output.
     - **Jumlah Neuron pada Setiap Hidden Layer** (2 nilai): Gunakan jumlah neuron yang sama pada setiap hidden layer dan lakukan eksperimen untuk dua jumlah neuron yang berbeda.
     - **Learning Rate** (0.01 dan 0.001).
     - **Epoch** (10, 100, 500): Jumlah iterasi.
     - **Activation Function** (2 jenis):
       - Yang pertama gunakan RELU pada semua hidden neuron dan sigmoid pada neuron output.
       - Yang kedua gunakan sigmoid untuk semua neuron.
   - Kombinasi nilai parameter ini akan memerlukan eksperimen untuk 48 kombinasi nilai parameter yang berbeda.

2. **Validasi dengan 10-Fold Cross Validation**
   - Untuk setiap kombinasi nilai seperti disebutkan dalam langkah pertama, jalankan model klasifikasi BPNN menggunakan *10-fold cross validation*.
   - Setiap kombinasi nilai parameter BPNN akan memberikan lima hasil validasi dari model.

3. **Pengukuran Kinerja Model**
   - Hitung akurasi, presisi, *recall*, F1-score, dan AUC ROC dari setiap data validasi pada langkah ke-2.

4. **Analisis Kinerja Model**
   - Lakukan analisis dari 10 hasil kinerja model BPNN untuk satu kombinasi parameter yang sedang diuji berdasarkan hasil pada langkah ke-3.

5. **Penentuan Model Terbaik**
   - Berdasarkan analisis pada langkah ke-4, tentukan kombinasi parameter yang Anda anggap menghasilkan model terbaik.
   - Jalankan model BPNN pada keseluruhan data pelatihan menggunakan kombinasi parameter tersebut dan hitung kinerja model (akurasi, presisi, *recall*, F1-score, dan AUC ROC).

6. **Ulangi untuk Kombinasi Nilai Berikutnya**
   - Ulangi langkah ke-2 hingga ke-5 untuk kombinasi nilai parameter BPNN berikutnya.

7. **Analisis Hasil Kinerja Model**
   - Lakukan analisis dari hasil kinerja kelima model BPNN yang diperoleh pada langkah ke-6.

8. **Prediksi dengan Model Terbaik**
   - Gunakan model BPNN terbaik untuk memprediksi semua record dari data tes.
   - Hitung akurasi, presisi, *recall*, F1-score, dan AUC ROC dari model terbaik pada data tes (data yang tidak digunakan dalam membangun model terbaik).


###7.1 Set up Stratified K-Fold Cross Validation

In [ ]:
# Set up Stratified K-Fold Cross Validation
n_splits = 10  # 10-fold
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Contoh cara membagi datanya
for fold, (train_index, val_index) in enumerate(skf.split(X_train, y_train)):
    print(f"Fold {fold+1}")
    print(f"Train size: {len(train_index)} | Validation size: {len(val_index)}")
    print("Distribusi Kelas di Validation Fold:")
    print(y_train.iloc[val_index].value_counts(normalize=True) * 100)
    print("-" * 40)

Fold 1
Train size: 205 | Validation size: 23
Distribusi Kelas di Validation Fold:
TARGET
1    52.173913
0    47.826087
Name: proportion, dtype: float64
----------------------------------------
Fold 2
Train size: 205 | Validation size: 23
Distribusi Kelas di Validation Fold:
TARGET
1    52.173913
0    47.826087
Name: proportion, dtype: float64
----------------------------------------
Fold 3
Train size: 205 | Validation size: 23
Distribusi Kelas di Validation Fold:
TARGET
1    52.173913
0    47.826087
Name: proportion, dtype: float64
----------------------------------------
Fold 4
Train size: 205 | Validation size: 23
Distribusi Kelas di Validation Fold:
TARGET
1    52.173913
0    47.826087
Name: proportion, dtype: float64
----------------------------------------
Fold 5
Train size: 205 | Validation size: 23
Distribusi Kelas di Validation Fold:
TARGET
0    52.173913
1    47.826087
Name: proportion, dtype: float64
----------------------------------------
Fold 6
Train size: 205 | Validation

###7.2 Kombinasi Parameter

In [ ]:
# Setup Stratified K-Fold
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Kombinasi parameter
hidden_layers = [1, 2]
neurons = [32, 64]
learning_rates = [0.01, 0.001]
epochs = [10, 100, 500]
activations = ['relu_sigmoid', 'sigmoid_sigmoid']

# Hasil eksperimen
results = []

# Scaling data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Mulai eksperimen
for layer in hidden_layers:
    for neuron in neurons:
        for lr in learning_rates:
            for epoch in epochs:
                for activation_combo in activations:
                    fold_metrics = {'accuracy': [], 'precision': [], 'recall': [], 'f1': [], 'auc': []}

                    print(f"Training BPNN: hidden_layers={layer}, neurons={neuron}, lr={lr}, epochs={epoch}, activation={activation_combo}")

                    for train_idx, val_idx in skf.split(X_train_scaled, y_train):
                        X_fold_train, X_fold_val = X_train_scaled[train_idx], X_train_scaled[val_idx]
                        y_fold_train, y_fold_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

                        # Build model
                        model = Sequential()
                        model.add(Input(shape=(X_fold_train.shape[1],)))

                        if activation_combo == 'relu_sigmoid':
                            activation_hidden = 'relu'
                            activation_output = 'sigmoid'
                        else:
                            activation_hidden = 'sigmoid'
                            activation_output = 'sigmoid'

                        model.add(Dense(neuron, activation=activation_hidden))
                        if layer == 2:
                            model.add(Dense(neuron, activation=activation_hidden))
                        model.add(Dense(1, activation=activation_output))

                        optimizer = Adam(learning_rate=lr)
                        model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

                        # Train model
                        y_fold_train = y_fold_train.astype(int)
                        model.fit(X_fold_train, y_fold_train.values, epochs=epoch, verbose=0, batch_size=32)


                        # Predict
                        y_pred_prob = model.predict(X_fold_val).flatten()
                        y_pred = (y_pred_prob > 0.5).astype(int)

                        # Handle NaN safe
                        if np.isnan(y_pred_prob).any():
                            auc = np.nan
                        else:
                            auc = roc_auc_score(y_fold_val, y_pred_prob)

                        # Hitung metrics
                        fold_metrics['accuracy'].append(accuracy_score(y_fold_val, y_pred))
                        fold_metrics['precision'].append(precision_score(y_fold_val, y_pred, zero_division=0))
                        fold_metrics['recall'].append(recall_score(y_fold_val, y_pred))
                        fold_metrics['f1'].append(f1_score(y_fold_val, y_pred))
                        fold_metrics['auc'].append(auc)

                    # Simpan hasil rata-rata dan std
                    result = {
                        'hidden_layers': layer,
                        'neurons': neuron,
                        'learning_rate': lr,
                        'epochs': epoch,
                        'activation': activation_combo,
                        'mean_accuracy': np.mean(fold_metrics['accuracy']),
                        'std_accuracy': np.std(fold_metrics['accuracy']),
                        'mean_precision': np.mean(fold_metrics['precision']),
                        'std_precision': np.std(fold_metrics['precision']),
                        'mean_recall': np.mean(fold_metrics['recall']),
                        'std_recall': np.std(fold_metrics['recall']),
                        'mean_f1': np.mean(fold_metrics['f1']),
                        'std_f1': np.std(fold_metrics['f1']),
                        'mean_auc': np.nanmean(fold_metrics['auc']),  # mean ignoring NaN
                        'std_auc': np.nanstd(fold_metrics['auc']),
                    }
                    results.append(result)

# Buat DataFrame hasil
results_df = pd.DataFrame(results)
display(results_df)

# Cari best model berdasarkan F1-score
best_model = results_df.sort_values(by='mean_f1', ascending=False).iloc[0]
print(f"\nBest BPNN Model: hidden_layers={best_model.hidden_layers}, neurons={best_model.neurons}, learning_rate={best_model.learning_rate}, epochs={best_model.epochs}, activation={best_model.activation}")

Training BPNN: hidden_layers=1, neurons=32, lr=0.01, epochs=10, activation=relu_sigmoid
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
Training BPNN: hidden_layers=1, neurons=32, lr=0.01, epochs=10, activation=sigmoid_sigmoid
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
Training BPNN: hidden_layers=1, neurons=32, lr=0.01, epochs=100, activation=relu_sigmoid
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━

,hidden_layers,neurons,learning_rate,epochs,activation,mean_accuracy,std_accuracy,mean_precision,std_precision,mean_recall,std_recall,mean_f1,std_f1,mean_auc,std_auc
0,1,32,0.010,10,relu_sigmoid,0.679447,0.122207,0.698013,0.143764,0.650758,0.165750,0.665280,0.135303,0.742080,0.111655
1,1,32,0.010,10,sigmoid_sigmoid,0.657905,0.093043,0.673161,0.121675,0.650000,0.081509,0.656589,0.087049,0.726309,0.102625
2,1,32,0.010,100,relu_sigmoid,0.644664,0.071397,0.651045,0.089028,0.651515,0.146938,0.641010,0.089242,0.684229,0.106130
3,1,32,0.010,100,sigmoid_sigmoid,0.640119,0.097559,0.637619,0.099675,0.668939,0.132524,0.647193,0.100674,0.727204,0.099022
4,1,32,0.010,500,relu_sigmoid,0.635573,0.069447,0.644525,0.085875,0.624242,0.135655,0.626021,0.089694,0.642631,0.085471
5,1,32,0.010,500,sigmoid_sigmoid,0.600988,0.089377,0.597306,0.096316,0.643939,0.145328,0.610865,0.111426,0.659573,0.109902
6,1,32,0.001,10,relu_sigmoid,0.631225,0.116531,0.637521,0.131337,0.615152,0.134405,0.623333,0.125398,0.684435,0.139431
7,1,32,0.001,10,sigmoid_sigmoid,0.662451,0.126098,0.675366,0.153225,0.660606,0.157598,0.659435,0.131193,0.723416,0.123064
8,1,32,0.001,100,relu_sigmoid,0.666403,0.114522,0.674755,0.120863,0.659848,0.164812,0.658873,0.124803,0.746281,0.106766
9,1,32,0.001,100,sigmoid_sigmoid,0.697431,0.100081,0.688554,0.106236,0.738636,0.135826,0.706508,0.102299,0.737948,0.110215



Best BPNN Model: hidden_layers=1, neurons=32, learning_rate=0.001, epochs=100, activation=sigmoid_sigmoid


###7.3 Model Terbaik

In [ ]:
# Bangun kembali model terbaik
model = Sequential()
model.add(Input(shape=(X_train_scaled.shape[1],)))

# Hidden layers
for _ in range(2):
    model.add(Dense(64, activation='sigmoid'))

# Output layer
model.add(Dense(1, activation='sigmoid'))

# Compile
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Fit model ke seluruh data training
model.fit(X_train_scaled, y_train, epochs=500, batch_size=32, verbose=1)

Epoch 1/500
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.4767 - loss: 0.6957
Epoch 2/500
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.4823 - loss: 0.6922  
Epoch 3/500
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5969 - loss: 0.6852 
Epoch 4/500
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6460 - loss: 0.6820
Epoch 5/500
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6498 - loss: 0.6776 
Epoch 6/500
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6503 - loss: 0.6750 
Epoch 7/500
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.6565 - loss: 0.6703
Epoch 8/500
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6476 - loss: 0.6673  
Epoch 9/500
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6144 - loss: 0.6619 
Epoch 10/500
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6012 - loss: 0.6630 
Epoch 11/500
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6669 - loss: 0.6573 
Epoch 12/500
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6616 -

###7.4 Hasil Akhir

In [ ]:
# Scaling X_test sesuai dengan scaler dari training
X_test_scaled = scaler.transform(X_test)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


y_test_pred = model.predict(X_test_scaled)
y_test_pred_label = (y_test_pred > 0.5).astype(int)
y_test = y_test.astype(int)

# Evaluasi
acc = accuracy_score(y_test, y_test_pred_label)
prec = precision_score(y_test, y_test_pred_label)
rec = recall_score(y_test, y_test_pred_label)
f1 = f1_score(y_test, y_test_pred_label)
auc = roc_auc_score(y_test, y_test_pred)

print(f"\nPerformance on Test Set:")
print(f"Accuracy : {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall   : {rec:.4f}")
print(f"F1-Score : {f1:.4f}")
print(f"AUC ROC  : {auc:.4f}")

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step

Performance on Test Set:
Accuracy : 0.6316
Precision: 0.6471
Recall   : 0.5789
F1-Score : 0.6111
AUC ROC  : 0.6932


## Video Presentasi

https://youtu.be/CaI7jXXSaZM